## Usage Examples

Train model and forecast future values with `predict` method.

In [ ]:
#| eval: false
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import matplotlib.pyplot as plt

from neuralforecast import NeuralForecast
from neuralforecast.utils import AirPassengersPanel, AirPassengersStatic
from neuralforecast.losses.pytorch import MAE
from neuralforecast.core import NeuralForecast
from neuralforecast.models import Informer, Autoformer, FEDformer, PatchTST, TSMixer

Y_train_df = AirPassengersPanel[AirPassengersPanel.ds<AirPassengersPanel['ds'].values[-12]].reset_index(drop=True) # 132 train
Y_test_df = AirPassengersPanel[AirPassengersPanel.ds>=AirPassengersPanel['ds'].values[-12]].reset_index(drop=True) # 12 test

model = TSMixer(h=12,
                input_size=24,
                n_series=2,
                # stat_exog_list=['airline1'],
                # futr_exog_list=['trend'],
                scaler_type='robust',
                max_steps=200,
                early_stop_patience_steps=-1,
                val_check_steps=10,
                learning_rate=1e-3,
                loss=MAE(),
                valid_loss=None,
                batch_size=32,
                n_block=2,
                ff_dim=64,
                target_slice=None
                )

fcst = NeuralForecast(models=[model], freq='M')
fcst.fit(df=Y_train_df, static_df=AirPassengersStatic, val_size=12)
forecasts = fcst.predict(futr_df=Y_test_df)

C:\Users\dhavy\AppData\Local\Temp\ipykernel_9008\3615986258.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was too old on your system - pyarrow 10.0.1 is the current minimum supported version as of this release.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
C:\Sapna\Nixtla_Project\neuralforecast\neuralforecast\utils.py:252: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  "ds": pd.date_range(start="1949-01-01", periods=len(AirPassengers), freq="M"),
C:\Sapna\Nixtla_Project\neuralforecast\neuralforecast\utils.py:264: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  pd.date_range(sta

Initializing input: (32, 24, 2)



Sanity Checking: |                                                                                      | 0/? …

Read the block: tensor([[[-1.0811, -1.0811],
         [-1.4595, -1.4595],
         [ 0.0270,  0.0270],
         [-0.1892, -0.1892],
         [ 0.0000,  0.0000],
         [ 1.8108,  1.8108],
         [ 2.9730,  2.9730],
         [ 3.0270,  3.0270],
         [ 1.3243,  1.3243],
         [-0.2162, -0.2162],
         [-1.3514, -1.3514],
         [-0.5135, -0.5135],
         [-0.4054, -0.4054],
         [-1.0000, -1.0000],
         [ 0.1892,  0.1892],
         [-0.1892, -0.1892],
         [ 0.2162,  0.2162],
         [ 2.1622,  2.1622],
         [ 3.6757,  3.6757],
         [ 4.0541,  4.0541],
         [ 1.3243,  1.3243],
         [ 0.1081,  0.1081],
         [-1.2162, -1.2162],
         [-0.4865, -0.4865]]])
torch.Size([24, 2]) 1
Input shape before temporal_linear: torch.Size([1, 24, 2])


RuntimeError: Given normalized_shape=[32, 24, 2], expected input with shape [*, 32, 24, 2], but got input of size[1, 24, 2]

In [ ]:
#| eval: false
# Plot quantile predictions
Y_hat_df = forecasts.reset_index(drop=False).drop(columns=['unique_id','ds'])
plot_df = pd.concat([Y_test_df, Y_hat_df], axis=1)
plot_df = pd.concat([Y_train_df, plot_df])

plot_df = plot_df[plot_df.unique_id=='Airline1'].drop('unique_id', axis=1)
plt.plot(plot_df['ds'], plot_df['y'], c='black', label='True')
plt.plot(plot_df['ds'], plot_df['StemGNN'], c='blue', label='Forecast')
plt.legend()
plt.grid()

In [ ]:
#| eval: false
fcst = NeuralForecast(models=[model], freq='M')
forecasts = fcst.cross_validation(df=AirPassengersPanel, static_df=AirPassengersStatic, n_windows=2, step_size=12)

In [ ]:
#| eval: false
# Plot quantile predictions
Y_hat_df = forecasts[forecasts['unique_id']=='Airline1']
Y_df = AirPassengersPanel[AirPassengersPanel['unique_id']=='Airline1']

plt.plot(Y_df['ds'], Y_df['y'], c='black', label='True')
plt.plot(Y_hat_df['ds'], Y_hat_df['StemGNN'], c='blue', label='Forecast')
plt.legend()
plt.grid()